# Homework 1 from Martin Gräf, Richard Baumann and Thomas Block

The main focus in this homework will be the ising model. In this particular example we look at a one-dimensional version of the modell, which describes a chain of spins. Due to this one-dimensionallity, if we observe one particle in the chain, it has only two neighbours. 

The Hamiltionian of the system can be written as: 

\begin{equation}
    H(\sigma) = - J \sum_{\langle x~y\rangle} \sigma_x \sigma_y - h \sum_x \sigma_x
\end{equation}





## Exercise 1: Discuss the physical meaning of J, in particular the sign of J, and the role it plays in magnets, for example.
In the equation above, J is a coupling constant, which decribes the interaction between the magnetic moments:

*  J<0: spins are aligned mostly antiparallel. The interaction is called antiferromagenitc.
*  J=0: spins are non-interacting.
*  J>0: spins are aligned mostly parallel. The interaction called ferromagenitc.

## Exercise 2: Clarify what it means to have periodic boundary conditions (nearest neighbors)



## Exercise 3: Implement the Ising 1d simulation: 

In [23]:
#just importing stuff, nothing to see here yet :)

import numpy as np 
import matplotlib.pyplot as plt
import math
import itertools 

As previously mentioned, the hamiltionian is calculated by:

\begin{equation}
    H(\sigma) = - J \sum_{\langle x~y\rangle} \sigma_x \sigma_y - h \sum_x \sigma_x.
\end{equation}

That can be implemented as a function.

In [2]:
#Calculate Hamiltonian. J and h are floats, the spin is supposed to be an array.
def hamiltionian(spin, j, h):
    minuend = 0
    subtrahend = 0
    for i in range(len(spin)-1):
        minuend = minuend + spin[i]*spin[i+1]
        subtrahend = subtrahend + spin[i] 
    return (-j*minuend-h*subtrahend)

We assume, that the spins are distributed following a Bolzmann distribution (a fact, that is given by the exercise sheet):

\begin{equation}
    P(s)=\frac{ exp \big[-\frac{H(s)}{k_{b}T} \big]}{\sum_{s`} exp \big[-\frac{H(s`)}{k_{b}T}\big]}  = \frac{1}{Z} exp \big[-\frac{H(s)}{k_{b}T} \big].
\end{equation}

Therefore Z can be determined via:

\begin{equation}
    Z={\sum_{s`} exp \big[-\frac{H(s`)}{k_{b}T}\big]} 
\end{equation}

In [8]:
def z_simulated(spin_configurations, j, h, t):
    sum=0
    for i in range(len(spin_configurations)):
        sum=sum+math.exp(-hamiltionian(spin_configurations[i], j, h)/t)
    return (sum)


#print(z_simulated([[1, 1, -1, -1, 1, 1, -1],[1, 1, -1, -1, 1, 1, -1]], 1, 1, 1))
print(len([[1, 1, -1, -1, 1, 1, -1],[1, 1, -1, -1, 1, 1, -1]]))

2


Last but not least, the spin arrays have to be generated, because this is the backbone of the exercise. The array, that is to be generated has to have a lenght of N. The contained values must be either 1 or -1. Good, that there is a function for that:

In [37]:
def gen_spinarrays(length):
    return(list(itertools.product([-1, 1], repeat=length)))


Now we can also implement the numerical solution that is given on the exercise sheet:

\begin{equation}
    Z=\lambda_{+}^{N}+\lambda_{-}^{N}, 
\end{equation}

with:

\begin{equation}
    \lambda_{\pm}=e^{\frac{J}{T}} \big( \cosh(\frac{h}{T}) \pm \sqrt{\sinh(\frac{h}{T})^{2}+ e^{-4\frac{J}{T}}} \big).
\end{equation}


In [38]:
print(z_simulated(gen_spinarrays(20), 1, 1, 1))

3.956744643262067e+16
